# Notebook to read FINN fire emissions

In [ ]:
import pandas as pd
import iris.pandas
import iris
import numpy as np
import iris.quickplot as qplt
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.io.img_tiles as cimgt
import datetime
from datetime import date
import pandas as pd


In [ ]:
species = 'PM25'

hires = False


# Path the FINN csv file
filepath = '/data/users/bdrummon/FINN/FINNv2.4/2018/MODIS_only/FINNv2.4_MOD_MOZART_2018_c20210617_subset.txt'

# Path to a template NAME file
if hires:
    gridpath = '/data/users/bdrummon/emissions/NAME/nameaq_emissions_2015_v1_regridded_2p2km/gridded_emissions_snap01_201501160000.nc'
else:
    gridpath = '/data/users/pmolina/AQ_Emissions/output/name/2015/std_ukdom_116x122/emep05/no_spread/gridded_emissions_snap01_201501160000.nc'

# Day of year - fire peaked 27th June, which is 178
days = {
    176 : 20180625, 
    177 : 20180626, 
    178 : 20180627,
    179 : 20180628, 
    180 : 20180629,
    181 : 20180630, 
    182 : 20180701, 
    183 : 20180702, 
    184 : 20180703
}

# Start date for emissions generation
dt_start = datetime.datetime(year=2018, month=6, day=14)
dt_end   = datetime.datetime(year=2018, month=7, day=17)

# Basic mask to cutout N England area
lat = (53.2, 54.8)
lon = (-3., -1.0)

# Map NAME species to FINN - mostly just different names for the same species
SPECIES_DICT = {
    'CO' : 'CO',
    'NO' : 'NO',
    'NO2' : 'NO2',
    'SO2' : 'SULPHUR-DIOXIDE',
    'NH3' : 'AMMONIA',
    'PM25' : 'PM25',
    'PMC' : 'PMC',
    'BIGENE' : 'BD', # Big alkenes lumped into 1,3-butadiene
    'C2H4' : 'C2H4',
    'C3H6' : 'C3H6',
    'CH2O' : 'HCHO', 
    'CH3CHO' : 'CH3CHO',
    'ISOP' : 'C5H8',
    'MGLY' : 'MGLYOX',
    'TOLUENE' : 'TOLUEN',
    'XYLENE' : 'OXYL'
}

NAMES = {
    'CO' : 'carbon monoxide',
    'NO' : 'nitrogen monoxide',
    'NO2' : 'nitrogen dioxide',
    'SULPHUR-DIOXIDE' : 'sulphur dioxide',
    'AMMONIA' : 'ammonia',
    'PM25' : 'pm2p5 dry aerosol particles',
    'PMC' : 'pm coarse dry aerosol particles',
    'BD' : '1,3-butadiene',
    'C2H4' : 'ethene',
    'C3H6' : 'propene',
    'HCHO' : 'formaldehyde',
    'CH3CHO' : 'acetaldehyde',
    'C5H8' : 'isoprene',
    'MGLYOX' : 'methyl glyoxal',
    'TOLUEN' : 'toluene',
    'OXYL' : 'o-xylene'
}

limits = {
    'PM25' : (0.8e-7, 2e-7)
}

PLOTDIR = '/home/h01/bdrummon/plots/Saddleworth_Moor/FINN/'

# Setup the map
mapstyle='street'
bkmap = cimgt.GoogleTiles(style=mapstyle)

In [ ]:
molar_masses = {
    ' APIN' : 136.23,
    'BENZENE' : 78.11,
    'BIGALK' : 72.151,
    'BIGENE' : 56.108, 
    'BPIN' : 136.238,
    'BZALD' : 106.124,
    'C2H2' : 26.038,
    'C2H4' : 28.051,
    'C2H6' : 30.070,
    'C3H6' : 42.081,
    'C3H8' : 44.097,
    'CH2O' : 30.026,
    'CH3CH2OH' : 46.069,
    'CH3CHO' : 44.053,
    'CH3CN' : 41.053,
    'CH3COCH3' : 58.080,
    'CH3COOH' : 60.052,
    'CH3OH' : 32.04,
    'CRESOL' : 108.13,
    'GLYALD' : 60.052,
    'HCN' : 27.0253,
    'HCOOH' : 46.025,
    'HONO' : 47.013,
    'HYAC' : 74.079,
    'ISOP' : 68.12,
    'LIMON' : 136.238,
    'MACR' : 70.09,
    'MEK' : 72.107,
    'MGLY' : 72.063,
    'MVK' : 70.09,
    'MYRC' : 136.238,
    'PHENOL' : 94.113,
    'TOLUENE' : 92.141,
    'XYLENE' : 106.168,
    'XYLOL' : 122.167,
    'CO' : 28.01,
    'NO' : 30.01,
    'NO2' : 46.0055,
    'SO2' : 64.066,
    'NH3' : 17.031
}


In [ ]:
#Load FINN data
df = pd.read_csv(filepath, index_col=False)
df

In [ ]:
# Load template emissions cube
cube = iris.load(gridpath)[0]
cube

In [ ]:
# Subet the dataframe to give a specific day (by day number) and between latitude and longitude limits
# Returns another dataframe
def subset_df(df, day):
    # Subset by day and area
    df_day = df[df['DAY']==day]
    df_area = df_day[df['LATI'] < lat[1]]
    df_area = df_area[df['LATI'] > lat[0]]
    df_area = df_area[df['LONGI'] < lon[1]]
    df_area = df_area[df['LONGI'] > lon[0]]

    return(df_area)

In [ ]:
# Put the fires into a grid - converting into per unit area
# Put fires into nearest cell
def grid_fires(df, cube, sp):
    # Create new cube using copy of template
    newcube = cube.copy()
    newcube.data = np.zeros(newcube.data.shape)

    gridlats = newcube.coord('latitude').points
    gridlons = newcube.coord('longitude').points

    # Loop over rows and assign each row to a cell in grid 
    # Emissions go into the nearest grid cell
    # This is additive - if multiple pixel fall within a cell
    for index, row in df.iterrows():
        firelat = row['LATI']
        firelon = row ['LONGI']

        idlat = (np.abs(gridlats - firelat)).argmin()
        idlon = (np.abs(gridlons - firelon)).argmin()

        # Convert to emission per unit area - now in kg or mole per day per m2
        # Fire emissions go into lowest level
        # Special case for PMC (coarse fraction of PM)
        # Need to calculate first from PM2.5 and PM10 : PMC = PM10 - PM2.5
        if sp == 'PMC':
            finn = row['PM10'] - row['PM25']
        else:
            finn = row[sp]
        
        newcube.data[0, idlat, idlon] =+ finn/row['AREA']
        
    # Convert units from mole/m2/day -> kg/m2/s
    newcube.data = newcube.data/24./60./60.

    # Gas only
    if sp not in ['PM25', 'PMC']:
        newcube.data = newcube.data*molar_masses[sp] * 1e-3
    
    # Update metadata
    newcube.attributes['tracer_name'] = SPECIES_DICT[sp]
    newcube.long_name = 'tendency of atmosphere mass content of '+NAMES[SPECIES_DICT[sp]]+' due to emission'
    newcube.var_name = SPECIES_DICT[sp]
    newcube.standard_name = None
    newcube.attributes['emiss_sector'] = 'snap11'
    newcube.attributes['source'] = 'FINN v2.4 MOZART-T1 speciation - Fire INventory from NCAR'
    newcube.attributes['title'] = 'NAME emissions generated from FINN database'
    newcube.attributes.pop('daily_scaling', None)
    newcube.attributes.pop('hourly_scaling', None)
    newcube.attributes.pop('vertical_scaling', None)
        
    return(newcube)

In [ ]:
# Script to plot fires with background map
def plot_fires(cube, df):

    # Replace zero with nans for plotting
    cube_nan = cube.copy()
    cube_nan.data[cube_nan.data == 0] = np.nan
    
    # Add FiNN pixels
    lats = []
    lons = []
    for index, row in df.iterrows():
        lats.append(row['LATI'])
        lons.append(row['LONGI'])    

    fig=plt.figure(figsize=(12,8), dpi= 100)
    ax = plt.axes(projection=bkmap.crs)
    ax.set_extent([-2.65,-1.85,53.35,53.75], ccrs.PlateCarree())
    qplt.pcolormesh(cube_nan[0], cmap='inferno', alpha=0.8, vmin = limits[species][0], vmax=limits[species][1])
    
    # Add fire pixels
    plt.scatter(np.asarray(lons), np.asarray(lats), marker='*', transform=ccrs.PlateCarree())
    
    # Add background map
    ax.add_image(bkmap, 10, interpolation='spline36')

In [ ]:
# Controlling script to make plots
for day in days:
    df_ss = subset_df(df, day)
    fires = grid_fires(df_ss, cube, species)
    plot_fires(fires, df_ss)
    plt.title('PM2.5 emissions, day: '+ str(day))
    plt.savefig(PLOTDIR+'pm25_emission_hires_day'+str(day)+'.png', dpi=150)
    plt.clf()
    #plt.show()
    #plt.clf()
    

## Generate NAME emission files


In [ ]:
days_full = pd.date_range(start=date(2018, 6, 15),end=date(2018, 7, 17)).to_pydatetime().tolist()

# Loop over days
for day in days_full:
    
    day_of_year = day.timetuple().tm_yday 
    
    
    # Get current day
    if day_of_year in days:
        day_subset = day_of_year
    else:
        day_subset = list(days.keys())[0]

    df_ss = subset_df(df, day_subset)

    cubelist = iris.cube.CubeList([])

    # Put data onto a grid and create cube
    for sp in SPECIES_DICT:

        fires = grid_fires(df_ss, cube, sp)

        # Update time
        fires.coord('time').convert_units("days since 2018-01-01")
        fires.coord('time').points = np.asarray(day_of_year)
        
        # If not in days then set data to zero
        if day_of_year not in days:
            fires.data = np.zeros(fires.data.shape)

        cubelist.append(fires)
    
    if hires:
        subdir = '2p2km'
    else:
        subdir = '12km'
        
    # Get date string
    dt = datetime.datetime(2018,1,1) + datetime.timedelta(day_of_year-1)
    dtstr = dt.strftime("%Y%m%d%H%M")
        
    iris.save(cubelist, 
              '/data/users/bdrummon/emissions/NAME/FINNv2.4_Saddleworth_Moor/'+subdir+'/gridded_emissions_wildfire_'+dtstr+'.nc')

## Calculate NMOC scaling factor
Derive from that converts NMOC emissions (kg/day) to species emissions (kg/day)

In [ ]:
# Plot specieated mass emissions for a cell
# Pick a row
row = df.iloc[15]

print(row['BZALD'])
print(row['BIGENE'])

# Convert to kg/day
mass_emissions = []
for species in molar_masses:
    mass_emissions.append(row[species]*molar_masses[species]*1e-3)
mass_emissions = np.asarray(mass_emissions)

# Sort by emission rate
zipped = zip(list(molar_masses.keys()), mass_emissions.tolist())
sort = sorted(zipped, key=lambda y: y[1])
names = []
emissions = []
for item in sort:
    names.append(item[0])
    emissions.append(item[1])
names = names[::-1]
emissions = emissions[::-1]


# Plot
plt.figure(figsize=(12,12))
x = np.arange(len(names))
plt.xticks(x, names, rotation='vertical')
plt.plot(x, emissions)
ax = plt.gca()
ax.xaxis.grid(True)

print(mass_emissions.sum())
print(row['NMOC'])
print(row['GENVEG'])
total = 0
for name, emission in zip(names, emissions):
    total += emission/row['NMOC']
    print(name, emission/row['NMOC'])
print(total)